## drf-access-policy

Django Rest Framework のthird party パッケージの drf-access-policy を使ってアクセス制御をする.

https://github.com/rsinger86/drf-access-policy

AWS の IAM の syntax と同じようにアクセス権限を定義できるっぽい. (AWS はよく知らないのでわからない)

###  環境

+ Django==2.2.4
+ djangorestframework==3.9.3

[sample code](https://github.com/fumuumuf/simple_drf/tree/access-policy)

## つかいかた

基本的な使い方は

1. `AccessPolicy` を継承したクラスを作成し,  statements を定義する
2.  view の `permission_classes` で作成したクラスを指定する

となる.([Example #1](https://github.com/rsinger86/drf-access-policy#example-1-policy-for-viewset))

### AccessPolicy

statements は

+ デフォルトはすべてのアクセスを拒否
+ 許可したいアクセスは `effect: allow` で許可する
+ ひとつでも `effect: deny` の statement にマッチすれば, そのアクセスは拒否される

という仕様. これに従い statement を定義する.


``` python
class ArticleAccessPolicy(AccessPolicy):
    """
    記事に関するアクセスポリシー

    Note:
        デフォルトはすべてのアクセスを拒否
    """

    statements = [
        {
            "action": ["list", "retrieve"],
            "principal": "*",
            "effect": "allow"
        },
        # editor グループに所属しているなら publish, unpublish アクションが可能
        {
            "action": ["publish", "unpublish"],
            "principal": ["group:editor"],
            "effect": "allow"
        },
        # is_author True なら delete(action:destroy) 可能
        {
            "action": ["destroy"],
            "principal": ["*"],
            "effect": "allow",
            "condition": "is_author"
        }
    ]

    def is_author(self, request, view, action) -> bool:
        """
        request.user が author であるか？
        """
        # scope_queryset を使用している場合, そのフィルタリングも適用されることに注意
        article = view.get_object()
        return request.user == article.author

    @classmethod
    def scope_queryset(cls, request, queryset):
        if request.user.groups.filter(name='editor').exists():
            return queryset

        return queryset.filter(status='publish')
```

### View の設定

view では `permission_classes` に作成した AccessPolicy を指定する.

`get_queryset` メソッドでは, AccessPolicy で定義しているクラスメソッド `scope_queryset` を呼び, アクセス権限に応じたフィルタリングを適用している.
AccessPolicy でルールとフィルタリングを一緒に定義することができるので分かりやすい. 

```python
class ArticleViewSet(viewsets.ModelViewSet):
    permission_classes = (ArticleAccessPolicy,)

    @property
    def access_policy(self):
        return self.permission_classes[0]

    serializer_class = ArticleSerializer
    queryset = Article.objects.select_related('category').prefetch_related('tags')

    def get_queryset(self):
        return self.access_policy.scope_queryset(
            self.request, Article.objects.all()
        )
```

## 使ってみる

実際にアクセスして検証する.

In [1]:
from rest_framework.test import APIClient
client = APIClient()

In [2]:
## editor グループを作成
editor_group,_ = Group.objects.get_or_create(name='editor')

In [3]:
author = User.objects.get(pk=1)
another = User.objects.get(pk=2)
reader = User.objects.get(pk=3)

# ユーザー を editor group に登録
editor_group.user_set.add(author)
editor_group.user_set.add(another)

## queryset のフィルタリング確認

`editor` group に所属しないユーザーは `unpublish`のデータを参照できないことを確認.

In [4]:
Article.objects.create(author=author, title='unpublish article', body='spam', status='unpublish')
Article.objects.create(author=author, title='published article', body='ham', status='publish')

<Article: 40 - published article>

In [5]:
### `editor` group に所属しないユーザーでのアクセス

In [6]:
client.force_authenticate(reader)
url = reverse('article-list')
client.get(url).json()

{'count': 1,
 'next': None,
 'previous': None,
 'results': [{'id': 40,
   'category': None,
   'title': 'published article',
   'body': 'ham',
   'status': 'publish',
   'author': 1,
   'tags': []}]}

In [7]:
### editor group に所属するユーザーでのアクセス

In [8]:
client.force_authenticate(author)
client.get(url).json()

{'count': 2,
 'next': None,
 'previous': None,
 'results': [{'id': 39,
   'category': None,
   'title': 'unpublish article',
   'body': 'spam',
   'status': 'unpublish',
   'author': 1,
   'tags': []},
  {'id': 40,
   'category': None,
   'title': 'published article',
   'body': 'ham',
   'status': 'publish',
   'author': 1,
   'tags': []}]}

## 削除機能の制限を確認

記事の author のユーザーのみ, 記事を削除できることを確認する.

In [9]:
# テスト用の記事を登録
article = Article(author=author, title='author-test', body='article body')
article.save()

### author でない他の editor が削除できないことを確認する

In [10]:
url = reverse('article-detail', kwargs={'pk': article.id})
client.force_authenticate(another)

In [11]:
res = client.delete(url)
print(res.status_code)
print(res.json())

Forbidden: /articles/41/


403
{'detail': 'このアクションを実行する権限がありません。'}


エラーコード 403 となりアクセスできない.

### author が削除できることを確認

In [12]:
client.force_authenticate(author)

In [13]:
res = client.delete(url)
print(res.status_code)

204


204 が返ってきてるので, 削除できている.

## まとめ

複数の action の制御をわかりやすく, まとめて定義できるので良い感じ ヽ(・ω・｡)ﾉ

他にも, グローバルに使用できる関数をつくれたり ([Re-Usable Conditions/Permissions](https://github.com/rsinger86/drf-access-policy#re-usable-conditionspermissions--)),  
statements を json に書いておいて読み込むこともできるみたい.([Loading Statements from External Source](https://github.com/rsinger86/drf-access-policy#loading-statements-from-external-source))

